In [ ]:
import json
import sys
import os
from pathlib import Path
import json 
from hotpotqa_utils import *
import pandas as pd
import numpy as np

sys.path.append('../')  

from utils import * 

from collections import Counter

/home/ehdtjr1220/miniconda3/envs/proj2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = json.load(open(f'../data/hotpotqa/hotpotqa.json'))
with open("react_prompt.json", 'r') as f:
    fewshot = json.load(f)

In [ ]:
for model in ['gpt-4o', 'gpt-4o-mini']:
    print(model)
    for shot_type in ['zero', 'few']:
        file_path = f"../result/hotpotqa/{model}/hotpotqa_{shot_type}.jsonl"
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
        else:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = [json.loads(line) for line in f]
            preds = [[] for _ in range(5)]
            for entry in data:
                for idx, out in enumerate(entry['model_outputs']):
                    preds[idx].append(extract_answer(out))


            em_scores = []
            f1_scores = []
            for i in range(5):
                em, f1 = get_em_f1(dataset, preds[i])
                em_scores.append(em.mean())
                f1_scores.append(f1.mean())


            avg_em = sum(em_scores) / len(em_scores)
            avg_f1 = sum(f1_scores) / len(f1_scores)
            print("{} Average EM: {:.4f} Average F1: {:.4f}".format(shot_type, avg_em, avg_f1))


print("llama")
for shot_type in ['zero', 'few']:
    file_path = f"../result/hotpotqa/llama/hotpotqa_{shot_type}.jsonl"
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
    else:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = [json.loads(line) for line in f]
        preds = [[] for _ in range(5)]
        for entry in data:
            for idx, out in enumerate(entry['resps'][0]):
                preds[idx].append(extract_answer(out))


        em_scores = []
        f1_scores = []
        for i in range(5):
            em, f1 = get_em_f1(dataset, preds[i])
            em_scores.append(em.mean())
            f1_scores.append(f1.mean())


        avg_em = sum(em_scores) / len(em_scores)
        avg_f1 = sum(f1_scores) / len(f1_scores)
        print("{} Average EM: {:.4f} Average F1: {:.4f}".format(shot_type, avg_em, avg_f1))

gpt-4o
zero Average EM: 0.3760 Average F1: 0.4988
few Average EM: 0.4464 Average F1: 0.5836
gpt-4o-mini
zero Average EM: 0.3148 Average F1: 0.4141
few Average EM: 0.3404 Average F1: 0.4513
llama
zero Average EM: 0.1520 Average F1: 0.2119
few Average EM: 0.1900 Average F1: 0.2514


In [ ]:
output_dir = "../likelihood/hotpotqa/"

subjects = [
    "gpt-4o/few", 
    "gpt-4o-mini/few", 
    "llama/few"
]

def add_pred(prob_type_filter, output_dir):
    output_dir = f"{output_dir}/{prob_type_filter}"
    likelihoods_file = os.path.join(output_dir, "all_likelihoods.json")
    if not os.path.exists(likelihoods_file):
        print(f"Error: {likelihoods_file} not found.")
        return
    with open(likelihoods_file, "r") as f:
        likelihoods = json.load(f)
    
    problem_groups = list(zip(*likelihoods))
    dataset = json.load(open(f'../data/hotpotqa/BM25/hotpotqa-bm25.json'))

    for problem_likelihoods in tqdm(problem_groups, desc="Processing problems"):
        problem_list = list(problem_likelihoods)
        for cl in problem_list: 
            pred = extract_answer(cl['model_output'])
            cl['pred'] = pred
            gt = dataset[cl['id']]['answers']
            cl['gt'] = gt

            em, f1 = get_em_f1([dataset[cl['id']]], [pred])
            cl['is_correct'] = (int(em[0]), float(f1[0]))


    with open(likelihoods_file, "w") as f:
        json.dump(likelihoods, f, indent=4)
    
    print(f"Updated file saved at: {likelihoods_file}")

In [4]:
for subject in subjects:
    add_pred(subject, output_dir)

Processing problems: 100%|██████████| 500/500 [00:01<00:00, 486.93it/s]


Updated file saved at: ../likelihood_qwen/hotpotqa//llama/few/all_likelihoods.json
